In [2]:
bedtools --version

bedtools v2.31.0


In [1]:
pwd

/Users/jmcdani/Documents/XY_benchmark/data_files_for_manuscript/chrXY_benchmark_exclusions


## chrXY draft benchmark error or unsure exclusions

#### Excluding no (error) and unsure manually curated variants 
Variants where the benchmark appears to be incorrect (no) or where NIST curators were unsure will be removed from draft benchmark prior to release.  This notebook describes the process of removing such regions.

Will exclude the entire entry from that stratification where an unsure/no variant  was. This ends up being akin to us stating “with this TR, which might be quite long, there was one site that looks like an error in the benchmark so to err on the side of trying to be correct far more often than not, we just went and excluded the entire TR”

1. create bed from curation tsv of no and unsure sites
2. expand bed regions modeling after what was done for CMRG, use the same GRCh38 TR and homopolymer file. Intention of doing multiintersect followed by merge is basically to take the union of bed files 
```
#expand bed coordinates to completely cover any overlapping homopolymers and tandem repeats
intersectBed -wa  
    -a resources/giab_stratifications/GRCh38_AllTandemRepeatsandHomopolymers_slop5.bed.gz  
    -b workflow/release_benchmark_generation/GRCh38_curation_medicalgene_smallvar_errorsorunsure.bed  
    | multiIntersectBed  
       -i stdin workflow/release_benchmark_generation/GRCh38_curation_medicalgene_smallvar_errorsorunsure.bed  
    | mergeBed  
       -i stdin  
    > workflow/release_benchmark_generation/GRCh38_curation_medicalgene_smallvar_errorsorunsure_repeatexpanded.bed
```
3. add 50bp slop on both sides

## chrY exclusions resulting in v1.0 benchmark bed

Sites received from J.Zook 7/21/23 (slack message) for the Y regions, I think it makes sense to exclude the entire segdup pairs:

1. for the inversion error, the pair of segdups is chrY:17,954,718-18,450,201 and chrY:17455804-17951260, and since these almost overlap we should exclude the entire region chrY:17455804-18450201.

this message sent to curators about the region:  The region on ~ chrY:17590000-18320000 has a cluster of putative FPs and FNs looks like it is related to a known inversion error in the T2T-Y HG002 assembly, which is corrected in the whole genome T2T-HG002. As the T2T-HG002 project progresses, we will track that region and consider how to improve an HG002 XY benchmark. As a note, it is interesting that ONT seems to support some variants in T2T-Y vs GRCh38, but it's probably related to challenges in mapping to these highly identical large segdups surrounding the small unique inverted sequence called P5 in the chrY paper. Reads aligned to the whole genome T2T-HG002 seem to support it as correct unlike T2T-Y. 

3. For the TSPY2 segdup pair, which is actually multiple segdups, we can exclude the segdup pairs in these regions: chrY:6234812-6532742 and chrY:9,628,425-9,919,592

it looks like it's somewhere inside the segdup pair chrY:6362052-6532742 and chrY:9,628,425-9,802,672 that mapping based calls start almost completely matching the assembly-based calls, so probably don't need to exclude more than this. It looks similar in both copies of the segdup, and the left side of the 9M segdup is near the start of the TSPY array, though  still end up including ~40kb between ~9.58-9.62M. Looking at other genes in the region we're excluding, TTTY22 also moves, with a 10kb del at 9.81M and 10kb ins at 6.36M. RBMY2NP and RMBY2GP and some other TTTY paralogs also swap positions. Interestingly, all of these genes are annotated differently on T2TY by CAT+liftoff and Refseq, where refseq seems to match gene sequences, whereas CAT+liftoff tries to match positions even if the gene sequences differ

message sent to curators:  particular pair of segdups separated by 4Mbp on chrY (chrY:9812675-9914616 and chrY:6260340-6362053,
which includes the TSPY2 gene) that was different between the assembly-based alignments and mapping-based alignments. We presented some slides to the T2T HG002 Q100 group about this region, as an example where the assembly seems correct but it could be aligned to GRCh38 in very different ways - https://docs.google.com/presentation/d/1NAwwM7_v633vEAxNjBH_IYDfqTwDJjRs2DDaDaZcCCY/edit?usp=sharing. After talking it through, I think we’ll exclude these regions from our current benchmark